Скрипт выполняет ручную дедупликацию названий районов на основе предопределенной карты DUPLICATE_MAP, которая сопоставляет неправильные названия с их каноническими версиями. Для каждой пары он сначала обновляет внешний ключ (big_folk_district_id) в таблице buildings, перенаправляя ссылки с дубликата на канонический ID, а затем удаляет устаревшую (дублирующую) запись из таблицы big_folk_districts, обеспечивая единообразие данных.

In [1]:
import sqlite3
import pandas as pd


DB_NAME = '../databases/dataset.db'
# Таблица, которую мы чистим (например, big_folk_districts или folk_districts)
DISTRICTS_TABLE = 'big_folk_districts' 
# Столбец в таблице 'buildings', который ссылается на DISTRICTS_TABLE
FOREIGN_KEY_COLUMN = 'big_folk_district_id' 

# Список дубликатов и канонических названий:
# { 'Неправильное_Имя': 'Каноническое_Имя', ... }
DUPLICATE_MAP = {
    'о. Русский': 'О. Русский',
    'о. Попова': 'О. Попова',
    'Вторая речка': 'Вторая Речка',
    '64 71 микрорайон': '64, 71 микрорайоны', # Выбираем одно из возможных написаний
    '3 рабочая': 'Третья рабочая' # Исправление опечатки
    # Если есть еще дубликаты 'Весенняя', добавьте их здесь, если ID разные
}

# 2. Установление соединения и определение функций
try:
    conn = sqlite3.connect(DB_NAME)
    cursor = conn.cursor()
    print(f"Успешное подключение к базе данных: {DB_NAME}")

    def get_id_by_name(name, table):
        """Возвращает ID по названию района."""
        query = f"SELECT id FROM {table} WHERE name = ?"
        cursor.execute(query, (name,))
        result = cursor.fetchone()
        return result[0] if result else None

    def consolidate_duplicates(duplicate_map, districts_table, fk_column):
        """Основная функция для объединения дубликатов."""
        print(f"\n--- Начинается консолидация в таблице {districts_table} ---")
        
        for wrong_name, correct_name in duplicate_map.items():
            
            # Получаем ID канонического и дублирующего района
            correct_id = get_id_by_name(correct_name, districts_table)
            wrong_id = get_id_by_name(wrong_name, districts_table)
            
            if not correct_id:
                print(f"❌ Ошибка: Не найден канонический ID для '{correct_name}'. Пропуск.")
                continue
            
            if not wrong_id:
                print(f"✅ Предупреждение: Не найден дубликат '{wrong_name}'. Возможно, уже исправлено. Пропуск.")
                continue

            print(f"\nОбработка: '{wrong_name}' (ID: {wrong_id}) -> '{correct_name}' (ID: {correct_id})")
            
            # 1. Обновление ссылок в таблице buildings
            update_query = f"""
            UPDATE buildings
            SET {fk_column} = ? 
            WHERE {fk_column} = ?
            """
            cursor.execute(update_query, (correct_id, wrong_id))
            print(f"   -> Обновлено ссылок в таблице buildings: {conn.total_changes}")

            # 2. Удаление дублирующей записи из таблицы районов
            delete_query = f"""
            DELETE FROM {districts_table} WHERE id = ?
            """
            cursor.execute(delete_query, (wrong_id,))
            print(f"   -> Удален дубликат из таблицы {districts_table}.")

        print("\n--- Консолидация завершена. ---")


    # 3. Выполнение очистки
    
    # 🚨 ВНИМАНИЕ: Запустите эту функцию, чтобы применить изменения.
    consolidate_duplicates(DUPLICATE_MAP, DISTRICTS_TABLE, FOREIGN_KEY_COLUMN)

    # 4. Сохранение изменений
    conn.commit()
    print("\nИзменения сохранены в базе данных.")

except sqlite3.Error as e:
    print(f"Произошла ошибка SQLite: {e}")
finally:
    if conn:
        conn.close()
        print("Соединение с БД закрыто.")

Успешное подключение к базе данных: ../databases/dataset.db

--- Начинается консолидация в таблице big_folk_districts ---

Обработка: 'о. Русский' (ID: 009af94d0a44e256f651e1f85d1b94e3) -> 'О. Русский' (ID: bbc3aa6c1924ba7aa571cb4d588355d3)
   -> Обновлено ссылок в таблице buildings: 839
   -> Удален дубликат из таблицы big_folk_districts.

Обработка: 'о. Попова' (ID: 16aa0efc7fbf0eacc272f84b771a8327) -> 'О. Попова' (ID: 807df8befe5a46cb79c5398a0760afc4)
   -> Обновлено ссылок в таблице buildings: 1084
   -> Удален дубликат из таблицы big_folk_districts.

Обработка: 'Вторая речка' (ID: 28f0c59576ad51c52bf864a76670abc2) -> 'Вторая Речка' (ID: 3caec8129b7983068459b162184fabe8)
   -> Обновлено ссылок в таблице buildings: 2634
   -> Удален дубликат из таблицы big_folk_districts.

Обработка: '64 71 микрорайон' (ID: b7cb6104dd1d8aa6c8752b879da04eee) -> '64, 71 микрорайоны' (ID: d3f85944a9834555a85268dd33c7a448)
   -> Обновлено ссылок в таблице buildings: 2651
   -> Удален дубликат из таблицы

Скрипт выполняет базовую очистку таблицы связей blackouts_buildings путём удаления всех записей, где внешний ключ blackout_id имеет значение NULL. Это обеспечивает целостность связи "отключение-здание", устраняя некорректные записи, которые не связаны ни с одним фактическим отключением.

In [ ]:
import sqlite3

# 1. Параметры подключения к базе данных
DB_NAME = '../databases/dataset.db'
TABLE_TO_CLEAN = 'blackouts_buildings'
COLUMN_TO_CHECK = 'blackout_id'

conn = None # Инициализируем соединение
try:
    # 2. Установление соединения
    conn = sqlite3.connect(DB_NAME)
    cursor = conn.cursor()
    print(f"Успешное подключение к базе данных: {DB_NAME}\n")
    
    # Сначала считаем, сколько записей будет удалено
    count_query = f"""
    SELECT COUNT(*) FROM {TABLE_TO_CLEAN} WHERE {COLUMN_TO_CHECK} IS NULL;
    """
    cursor.execute(count_query)
    records_to_delete = cursor.fetchone()[0]
    
    if records_to_delete == 0:
        print(f"✅ В таблице {TABLE_TO_CLEAN} не найдено записей с {COLUMN_TO_CHECK} = NULL. Очистка не требуется.")
    else:
        print(f"Обнаружено записей для удаления: {records_to_delete}")
        
        # 3. Выполнение запроса на удаление
        delete_query = f"""
        DELETE FROM {TABLE_TO_CLEAN} WHERE {COLUMN_TO_CHECK} IS NULL;
        """
        cursor.execute(delete_query)
        
        # 4. Сохранение изменений
        conn.commit()
        
        print(f"🗑️ Успешно удалено {conn.total_changes} записей из таблицы {TABLE_TO_CLEAN}.")
        print("\nИзменения сохранены в базе данных.")

except sqlite3.Error as e:
    print(f"❌ Произошла ошибка SQLite: {e}")
finally:
    # 5. Закрытие соединения
    if conn:
        conn.close()
        print("Соединение с БД закрыто.")

Успешное подключение к базе данных: ../databases/dataset.db

Обнаружено записей для удаления: 1243966
🗑️ Успешно удалено 1243966 записей из таблицы blackouts_buildings.

Изменения сохранены в базе данных.
Соединение с БД закрыто.


Скрипт проводит комплексную диагностику чистоты данных в таблице streets, проверяя на NULL-значения и настоящие дубликаты (одинаковое название улицы в пределах одного города после нормализации). Дополнительно он проверяет ссылочную целостность, выявляя здания в таблице buildings, которые ссылаются на несуществующие street_id, предоставляя рекомендации для устранения всех найденных проблем.

In [3]:
import sqlite3
import pandas as pd

DB_NAME = '../databases/dataset.db'
STREETS_TABLE = 'streets'
BUILDINGS_TABLE = 'buildings'
CITIES_TABLE = 'cities'

conn = None
try:
    conn = sqlite3.connect(DB_NAME)
    cursor = conn.cursor()

    # --- Подготовка: Получение справочника городов для удобного вывода ---
    df_cities = pd.read_sql_query(f"SELECT id, name FROM {CITIES_TABLE}", conn)
    city_map = df_cities.set_index('id')['name'].to_dict()
    print(f"Успешное подключение к базе данных: {DB_NAME}")
    print(f"Обнаруженные города: {list(city_map.values())}")
    
    # Загрузка таблицы streets в DataFrame
    df_streets = pd.read_sql_query(f"SELECT id, name, city_id FROM {STREETS_TABLE}", conn)
    
    
    # ----------------------------------------------------------------------
    ## 1. Проверка на NULL в ключевых полях
    # ----------------------------------------------------------------------
    print("\n" + "="*50)
    print("## 1. Проверка на NULL в streets")
    print("="*50)
    
    null_counts = df_streets.isnull().sum()
    
    if null_counts.sum() == 0:
        print("✅ NULL не обнаружено ни в одном столбце.")
    else:
        print("❌ Обнаружены NULL:")
        print(null_counts[null_counts > 0])
        print("Рекомендация: записи с пустым 'id' или 'name' следует удалить.")

    
    # ----------------------------------------------------------------------
    ## 2. Проверка НАСТОЯЩИХ дубликатов (Одинаковое название И Город)
    # ----------------------------------------------------------------------
    print("\n" + "="*50)
    print("## 2. Проверка НАСТОЯЩИХ дубликатов")
    print("   (Одинаковое название в ОДНОМ городе)")
    print("="*50)
    
    # Нормализуем название улицы для поиска дубликатов без учета регистра/пробелов
    df_streets['normalized_name'] = df_streets['name'].str.lower().str.strip()
    
    # Ищем дубликаты, которые имеют одинаковое нормализованное название И одинаковый city_id
    duplicate_streets = df_streets.duplicated(subset=['normalized_name', 'city_id'], keep=False)
    duplicate_names = df_streets[duplicate_streets]
    
    if duplicate_names.empty:
        print("✅ Потенциальные дубликаты (одинаковое название в ОДНОМ городе) не обнаружены. Таблица чиста!")
    else:
        # Группируем, чтобы увидеть дублирующиеся пары
        duplicates_grouped = duplicate_names.groupby(['normalized_name', 'city_id'])[['id', 'name']].agg(list)
        duplicates_grouped['ID_Count'] = duplicates_grouped['id'].apply(len)
        
        print(f"❌ Обнаружено {len(duplicates_grouped)} пар, которые являются НАСТОЯЩИМИ дубликатами.")
        
        # Выводим результат с названием города
        output_df = duplicates_grouped.reset_index().drop(columns=['normalized_name'])
        output_df['City_Name'] = output_df['city_id'].map(city_map)
        
        print(output_df[['City_Name', 'name', 'ID_Count']])
        
        print("\nРекомендация: Выберите один канонический ID для каждого дубликата и обновите ссылки в 'buildings', затем удалите дубликаты из 'streets'.")
        
    
    # ----------------------------------------------------------------------
    ## 3. Проверка распределения по городам (city_id)
    # ----------------------------------------------------------------------
    print("\n" + "="*50)
    print("## 3. Проверка распределения по городам")
    print("="*50)

    # Группируем улицы по city_id
    city_counts = df_streets['city_id'].map(city_map).value_counts(dropna=False)
    
    print("Распределение улиц по городам:")
    print(city_counts)
    
    # Находим ID Владивостока
    vladivostok_id = df_cities[df_cities['name'].str.contains('Владивосток', case=False)]['id'].iloc[0] if not df_cities[df_cities['name'].str.contains('Владивосток', case=False)].empty else None
    


    # ----------------------------------------------------------------------
    ## 4. Проверка ссылочной целостности: Buildings -> Streets
    # ----------------------------------------------------------------------
    print("\n" + "="*50)
    print("## 4. Проверка ссылочной целостности: Buildings -> Streets")
    print("="*50)
    
    # Находим buildings, которые ссылаются на несуществующий street_id
    orphan_query = f"""
    SELECT 
        COUNT(T1.id) 
    FROM {BUILDINGS_TABLE} AS T1
    LEFT JOIN {STREETS_TABLE} AS T2 ON T1.street_id = T2.id
    WHERE T2.id IS NULL AND T1.street_id IS NOT NULL;
    """
    
    cursor.execute(orphan_query)
    orphan_count = cursor.fetchone()[0]
    
    if orphan_count == 0:
        print("✅ Нет зданий, ссылающихся на несуществующие street_id.")
    else:
        print(f"❌ Обнаружено {orphan_count} зданий с недействительным street_id.")
        print("Рекомендация: Эти 'street_id' в таблице buildings следует либо обнулить, либо найти/создать соответствующие записи в таблице streets.")


except sqlite3.Error as e:
    print(f"❌ Произошла ошибка SQLite: {e}")
finally:
    if conn:
        conn.close()
        print("\nСоединение с БД закрыто.")

Успешное подключение к базе данных: ../databases/dataset.db
Обнаруженные города: ['Артем', 'Владивосток']

## 1. Проверка на NULL в streets
✅ NULL не обнаружено ни в одном столбце.

## 2. Проверка НАСТОЯЩИХ дубликатов
   (Одинаковое название в ОДНОМ городе)
✅ Потенциальные дубликаты (одинаковое название в ОДНОМ городе) не обнаружены. Таблица чиста!

## 3. Проверка распределения по городам
Распределение улиц по городам:
city_id
Владивосток    1201
Артем           822
Name: count, dtype: int64

## 4. Проверка ссылочной целостности: Buildings -> Streets
✅ Нет зданий, ссылающихся на несуществующие street_id.

Соединение с БД закрыто.


Скрипт выполняет дедупликацию названий районов в таблице folk_districts путём сопоставления дубликатов (например, разных написаний одного и того же места) с каноническими именами с использованием регулярных выражений и нормализации. После идентификации дубликатов, он обновляет соответствующие внешние ключи в таблице buildings и удаляет лишние записи из folk_districts, обеспечивая целостность и чистоту данных.

In [4]:
import sqlite3
import pandas as pd
import re

# 1. Параметры подключения
DB_NAME = '../databases/dataset.db'
FD_TABLE = 'folk_districts'
FOREIGN_KEY_COLUMN = 'folk_district_id' 

# 🚨 Канонические названия, которые ДОЛЖНЫ остаться
CANONICAL_NAMES = [
    '64, 71 микрорайон',
    'О. Попова',
    'о. Рейнеке',
    'О. Русский',
    'П-ов. Песчаный',  # ← Каноническое имя для всех вариантов!
    'Район Хребта Богатая Грива',
    'Тавайза',
    'Угловое'
]

# 🔥 СПЕЦИАЛЬНЫЕ ГРУППЫ ДЛЯ СЛОЖНЫХ СЛУЧАЕВ
SPECIAL_GROUPS = {
    'Песчаный': {
        'canonical': 'П-ов. Песчаный',
        'patterns': [
            r'п[\s\-]*ов\.?\s*песчан',  # п-ов. Песчаный, пов. Песчаный
            r'полуостров\s*песчан',      # полуостров Песчаный
            r'по[\s\-]*ов\.?\s*песчан',  # по-ов. Песчаный
            r'песчан.*ый',               # Песчаный, Песчанный
        ]
    }
}

conn = None
try:
    conn = sqlite3.connect(DB_NAME)
    cursor = conn.cursor()
    print(f"✅ Успешное подключение к базе данных: {DB_NAME}")
    
    # Загружаем все данные для программной обработки
    df_fd = pd.read_sql_query(f"SELECT id, name FROM {FD_TABLE}", conn)
    print(f"📊 Загружено записей из {FD_TABLE}: {len(df_fd)}")
    
    # ----------------------------------------------------------------------
    # 2. Формирование карты замены ID - УЛУЧШЕННАЯ ВЕРСИЯ
    # ----------------------------------------------------------------------
    ID_REPLACEMENT_MAP = {}
    total_duplicates_found = 0
    
    # Сначала обработаем специальные группы
    for group_name, group_config in SPECIAL_GROUPS.items():
        canonical_name = group_config['canonical']
        patterns = group_config['patterns']
        
        print(f"\n🔍 Обрабатываем специальную группу: '{group_name}'")
        print(f"   Каноническое имя: '{canonical_name}'")
        
        # Находим канонический ID
        canonical_row = df_fd[df_fd['name'] == canonical_name]
        
        if canonical_row.empty:
            print(f"❌ Ошибка: Не найден канонический ID для '{canonical_name}'. Пропуск группы.")
            continue
            
        correct_id = canonical_row['id'].iloc[0]
        print(f"   Канонический ID: {correct_id}")
        
        # Ищем все совпадения по паттернам
        matching_ids = []
        for pattern in patterns:
            compiled_pattern = re.compile(pattern, re.IGNORECASE)
            matches = df_fd[df_fd['name'].str.contains(compiled_pattern, na=False)]
            
            for _, row in matches.iterrows():
                if row['id'] != correct_id and row['id'] not in matching_ids:
                    matching_ids.append(row['id'])
                    print(f"   🎯 Найден дубликат по паттерну '{pattern}': '{row['name']}' (ID: {row['id']})")
        
        # Добавляем в карту замены
        for wrong_id in matching_ids:
            ID_REPLACEMENT_MAP[wrong_id] = correct_id
        
        duplicates_count = len(matching_ids)
        total_duplicates_found += duplicates_count
        print(f"   ✅ Найдено {duplicates_count} дубликатов: {matching_ids}")
    
    # Затем обрабатываем остальные канонические имена (простая проверка по точному совпадению)
    for canonical_name in CANONICAL_NAMES:
        # Пропускаем те, что уже обработаны в специальных группах
        if any(canonical_name == group_config['canonical'] for group_config in SPECIAL_GROUPS.values()):
            continue
            
        print(f"\n🔍 Обрабатываем: '{canonical_name}'")
        
        # Находим канонический ID
        canonical_row = df_fd[df_fd['name'] == canonical_name]
        
        if canonical_row.empty:
            print(f"❌ Ошибка: Не найден канонический ID для '{canonical_name}'. Пропуск.")
            continue
            
        correct_id = canonical_row['id'].iloc[0]
        
        # Для остальных используем простую нормализацию
        normalized_key = (
            canonical_name.lower()
            .replace('.', '').replace(',', '').replace(' ', '')
            .replace('-', '')
        )
        
        # Создаем normalized_key для всех записей (только для этого имени)
        temp_normalized = (
            df_fd['name'].str.lower()
            .str.replace(r'[.,\-\s+]', '', regex=True)
        )
        
        group_rows = df_fd[temp_normalized == normalized_key]
        ids_to_delete = group_rows[group_rows['id'] != correct_id]['id'].tolist()
        
        if not ids_to_delete:
            print(f"   ✅ Дубликатов не найдено.")
            continue
            
        # Заполняем карту замены
        for wrong_id in ids_to_delete:
            ID_REPLACEMENT_MAP[wrong_id] = correct_id
        
        duplicates_count = len(ids_to_delete)
        total_duplicates_found += duplicates_count
        print(f"   ✅ Найдено {duplicates_count} дубликатов → {ids_to_delete}")
    
    print(f"\n📈 ИТОГО: Найдено {total_duplicates_found} дубликатов для замены!")
    print(f"📋 Карта замены: {ID_REPLACEMENT_MAP}")

    if not ID_REPLACEMENT_MAP:
        print("\n🎉 Дубликатов не найдено! База уже чистая.")
    else:
        # ----------------------------------------------------------------------
        # 3. Выполнение замены в buildings и удаление из folk_districts
        # ----------------------------------------------------------------------
        print("\n🔄 --- Выполнение замены и удаления в БД ---")
        total_buildings_updated = 0
        
        for wrong_id, correct_id in ID_REPLACEMENT_MAP.items():
            print(f"  🔧 Обрабатываем ID {wrong_id} → {correct_id}")
            
            # A. Обновление ссылок в таблице buildings
            update_query = f"""
            UPDATE buildings
            SET {FOREIGN_KEY_COLUMN} = ? 
            WHERE {FOREIGN_KEY_COLUMN} = ?
            """
            cursor.execute(update_query, (correct_id, wrong_id))
            updated_count = cursor.rowcount
            total_buildings_updated += updated_count
            print(f"     📍 Обновлено зданий: {updated_count}")
            
            # B. Удаление дублирующей записи
            delete_query = f"""
            DELETE FROM {FD_TABLE} WHERE id = ?
            """
            cursor.execute(delete_query, (wrong_id,))
            
            if cursor.rowcount == 0:
                print(f"     ⚠️  ID {wrong_id} уже удален ранее")
            else:
                print(f"     🗑️  Удален дубликат ID {wrong_id}")
        
        # 4. Сохранение изменений
        conn.commit()
        print(f"\n🎉 ✅ ИТОГО:")
        print(f"   📍 Обновлено ссылок в buildings: {total_buildings_updated}")
        print(f"   🗑️  Удалено дубликатов из {FD_TABLE}: {len(ID_REPLACEMENT_MAP)}")
        print("   💾 Изменения СОХРАНЕНЫ в базе данных!")

except sqlite3.Error as e:
    print(f"❌ Произошла ошибка SQLite: {e}")
    if conn:
        conn.rollback()
        print("   🔄 Изменения отменены (rollback).")
except Exception as e:
    print(f"❌ Неожиданная ошибка: {e}")
    if conn:
        conn.rollback()
        print("   🔄 Изменения отменены (rollback).")
finally:
    if conn:
        conn.close()
        print("🔌 Соединение с БД закрыто.")

✅ Успешное подключение к базе данных: ../databases/dataset.db
📊 Загружено записей из folk_districts: 3594

🔍 Обрабатываем специальную группу: 'Песчаный'
   Каноническое имя: 'П-ов. Песчаный'
   Канонический ID: 2b045aed62a3ca490fde36fdad2b78f2
   🎯 Найден дубликат по паттерну 'п[\s\-]*ов\.?\s*песчан': 'п-ов Песчаный' (ID: 496a98094feab6c1daf1c6803dcfde7e)
   🎯 Найден дубликат по паттерну 'по[\s\-]*ов\.?\s*песчан': 'По-ов. Песчанный' (ID: 11e0f52adfe4a253cf3cc1d48dd12802)
   🎯 Найден дубликат по паттерну 'по[\s\-]*ов\.?\s*песчан': 'По-ов. Песчанный' (ID: 39a7baf6077f13db7888cf7a82abeff2)
   🎯 Найден дубликат по паттерну 'по[\s\-]*ов\.?\s*песчан': 'По-ов. Песчанный' (ID: 2c4e3fe794239840f5d2897e0fe8a500)
   🎯 Найден дубликат по паттерну 'по[\s\-]*ов\.?\s*песчан': 'По-ов. Песчанный' (ID: f4edc4a3567f88dd657841c099200682)
   🎯 Найден дубликат по паттерну 'по[\s\-]*ов\.?\s*песчан': 'По-ов. Песчанный' (ID: fa547163c8738a3e4dd3400e15b4e5fb)
   ✅ Найдено 6 дубликатов: ['496a98094feab6c1daf1c68

Скрипт проводит углублённый аудит чистоты данных в таблице buildings, фокусируясь исключительно на реальных зданиях (is_fake = 0). Он выполняет три ключевые проверки: 
1) выявляет NULL-значения в важных полях (street_id, number, coordinates, folk_district_id);
2) ищет полные дубликаты по связке "улица + номер + координаты";
3) подсчитывает здания-сироты — реальные здания, которые никогда не были связаны с записями об отключениях в таблице blackouts_buildings.

In [5]:
import sqlite3
import pandas as pd

DB_NAME = '../databases/dataset.db'
BUILDINGS_TABLE = 'buildings'
BB_TABLE = 'blackouts_buildings' # Таблица отключений

conn = None
try:
    conn = sqlite3.connect(DB_NAME)
    cursor = conn.cursor()
    
    # ----------------------------------------------------------------------
    # 0. Подсчет и фильтрация фейковых зданий
    # ----------------------------------------------------------------------
    cursor.execute(f"SELECT COUNT(*) FROM {BUILDINGS_TABLE}")
    total_buildings_all = cursor.fetchone()[0]
    
    # Подсчет фейковых зданий
    cursor.execute(f"SELECT COUNT(*) FROM {BUILDINGS_TABLE} WHERE is_fake = 1")
    fake_buildings_count = cursor.fetchone()[0]
    
    # Определяем реальное количество зданий для анализа
    total_buildings_real = total_buildings_all - fake_buildings_count
    
    print(f"Успешное подключение к базе данных: {DB_NAME}")
    print(f"Всего записей в таблице {BUILDINGS_TABLE}: {total_buildings_all}")
    print(f"**❗️ ФЕЙКОВЫЕ ЗДАНИЯ (is_fake = 1): {fake_buildings_count} ({fake_buildings_count/total_buildings_all:.2%})**")
    print(f"**🟢 ЗДАНИЯ ДЛЯ АНАЛИЗА (Реальные): {total_buildings_real}**")

    # Переменная для использования в расчетах процентов
    denominator = total_buildings_real if total_buildings_real > 0 else total_buildings_all 

    # ----------------------------------------------------------------------
    print("\n" + "="*50)
    print("## 1. Проверка на NULL (Только для РЕАЛЬНЫХ зданий)")
    print("="*50)

    # 1. Проверка на NULL в ключевых полях, ИСКЛЮЧАЯ ФЕЙКОВЫЕ ЗДАНИЯ
    null_query = f"""
    SELECT 
        SUM(CASE WHEN street_id IS NULL THEN 1 ELSE 0 END) AS null_street_id,
        SUM(CASE WHEN number IS NULL THEN 1 ELSE 0 END) AS null_number,
        SUM(CASE WHEN coordinates IS NULL THEN 1 ELSE 0 END) AS null_coordinates, 
        SUM(CASE WHEN folk_district_id IS NULL THEN 1 ELSE 0 END) AS null_folk_district_id
    FROM {BUILDINGS_TABLE}
    WHERE is_fake = 0;
    """
    cursor.execute(null_query)
    null_counts = dict(zip(['street_id', 'number', 'coordinates', 'folk_district_id'], cursor.fetchone()))
    
    for col, count in null_counts.items():
        if count > 0:
            print(f"❌ Столбец '{col}': Обнаружено {count} NULL ({count/denominator:.2%} от реальных)")
        else:
            print(f"✅ Столбец '{col}': NULL не обнаружено среди реальных зданий.")

    # ----------------------------------------------------------------------
    print("\n" + "="*50)
    print("## 2. Проверка на полные дубликаты (Только для РЕАЛЬНЫХ)")
    print("="*50)

    # 2. Поиск дубликатов по Улице + Номеру + Координатам, ИСКЛЮЧАЯ ФЕЙКОВЫЕ ЗДАНИЯ
    dup_query = f"""
    SELECT 
        T1.name AS street_name, T2.number, T2.coordinates, COUNT(*) as count
    FROM {BUILDINGS_TABLE} AS T2
    LEFT JOIN streets AS T1 ON T2.street_id = T1.id
    WHERE T2.is_fake = 0
    GROUP BY T2.street_id, T2.number, T2.coordinates
    HAVING count > 1
    LIMIT 10;
    """
    df_duplicates = pd.read_sql_query(dup_query, conn)
    
    if df_duplicates.empty:
        print("✅ Полных дубликатов среди реальных зданий не обнаружено.")
    else:
        print(f"❌ Обнаружены потенциальные полные дубликаты среди реальных зданий (показаны первые 10):")
        print(df_duplicates)
        print("\nРекомендация: Эти записи следует объединить, сохранив только один ID.")

    # ----------------------------------------------------------------------
    print("\n" + "="*50)
    print("## 3. Проверка на здания-сироты (Нет связи с отключениями)")
    print("="*50)
    
    # 3. Подсчет зданий (реальных), которые никогда не фигурировали в отключениях
    orphan_query = f"""
    SELECT COUNT(T1.id)
    FROM {BUILDINGS_TABLE} AS T1
    LEFT JOIN {BB_TABLE} AS T2 ON T1.id = T2.building_id
    WHERE T2.building_id IS NULL AND T1.is_fake = 0;
    """
    cursor.execute(orphan_query)
    orphan_count = cursor.fetchone()[0]
    
    if orphan_count == 0:
        print("✅ Каждое реальное здание имеет хотя бы одну запись об отключении.")
    else:
        print(f"ℹ️ Обнаружено {orphan_count} реальных зданий, которые НЕ фигурируют в таблице отключений.")
        print(f"   Это {orphan_count/denominator:.2%} от общего числа реальных зданий.")
        print("   (Это нормально, если в базе есть здания, по которым отключений не было).")

except sqlite3.Error as e:
    print(f"❌ Произошла ошибка SQLite: {e}")
finally:
    if conn:
        conn.close()
        print("\nСоединение с БД закрыто.")

Успешное подключение к базе данных: ../databases/dataset.db
Всего записей в таблице buildings: 58199
**❗️ ФЕЙКОВЫЕ ЗДАНИЯ (is_fake = 1): 12080 (20.76%)**
**🟢 ЗДАНИЯ ДЛЯ АНАЛИЗА (Реальные): 46119**

## 1. Проверка на NULL (Только для РЕАЛЬНЫХ зданий)
✅ Столбец 'street_id': NULL не обнаружено среди реальных зданий.
✅ Столбец 'number': NULL не обнаружено среди реальных зданий.
✅ Столбец 'coordinates': NULL не обнаружено среди реальных зданий.
❌ Столбец 'folk_district_id': Обнаружено 24672 NULL (53.50% от реальных)

## 2. Проверка на полные дубликаты (Только для РЕАЛЬНЫХ)
✅ Полных дубликатов среди реальных зданий не обнаружено.

## 3. Проверка на здания-сироты (Нет связи с отключениями)
ℹ️ Обнаружено 26191 реальных зданий, которые НЕ фигурируют в таблице отключений.
   Это 56.79% от общего числа реальных зданий.
   (Это нормально, если в базе есть здания, по которым отключений не было).

Соединение с БД закрыто.
